In [1]:
import wrangle

import pandas as pd
import numpy as np

import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.float_format = '{:20,.2f}'.format

In [2]:
df2 = wrangle.get_zillow()

In [3]:
df2 = wrangle.single_family(df2)

In [4]:
df2.bedrooms.value_counts()

 3.00    23296
 4.00    15202
 2.00     8314
 5.00     3964
 6.00      633
 1.00      610
 7.00      105
 8.00       23
 9.00        8
10.00        2
11.00        1
Name: bedrooms, dtype: int64

In [5]:
df2.shape

(52158, 58)

In [6]:
df = wrangle.wrangle_zillow()

DataFrame head: 

   parcelid  basementsqft            bathrooms             bedrooms  \
0  14297519           NaN                 3.50                 4.00   
1  17052889           NaN                 1.00                 2.00   
2  14186244           NaN                 2.00                 3.00   

     calculatedbathnbr  finishedfloor1squarefeet  \
0                 3.50                       NaN   
1                 1.00                  1,465.00   
2                 2.00                       NaN   

   calculatedfinishedsquarefeet  finishedsquarefeet12  finishedsquarefeet13  \
0                      3,100.00              3,100.00                   NaN   
1                      1,465.00              1,465.00                   NaN   
2                      1,243.00              1,243.00                   NaN   

   finishedsquarefeet15  ...   censustractandblock             logerror  \
0                   NaN  ... 60,590,630,072,012.00                 0.03   
1                   N

In [7]:
df.shape

(52158, 30)

In [8]:
def summarize(df):
    print('DataFrame head: \n')
    print(df.head(3))
    print('--------------')
    print('Shape:   ', df.shape)
    print('---------------')
    print('Info:    \n')
    df.info()
    print('---------------')
    print(df.describe())


In [9]:
summarize(df)

DataFrame head: 

   parcelid            bathrooms             bedrooms    calculatedbathnbr  \
0  14297519                 3.50                 4.00                 3.50   
1  17052889                 1.00                 2.00                 1.00   
2  14186244                 2.00                 3.00                 2.00   

   calculatedfinishedsquarefeet  finishedsquarefeet12                 fips  \
0                      3,100.00              3,100.00             6,059.00   
1                      1,465.00              1,465.00             6,111.00   
2                      1,243.00              1,243.00             6,059.00   

           fullbathcnt             latitude            longitude  ...  \
0                 3.00        33,634,931.00      -117,869,207.00  ...   
1                 1.00        34,449,266.00      -119,281,531.00  ...   
2                 2.00        33,886,168.00      -117,823,170.00  ...   

   structuretaxvaluedollarcnt            tax_value       assess

In [10]:
def nulls_by_row(df):
    num_missing = df.isnull().sum(axis=1)
    prcnt_miss = num_missing / df.shape[1] * 100
    
    rows_missing = pd.DataFrame({'num_cols_missing': num_missing, 'percent_cols_missing' : prcnt_miss})
    rows_missing = rows_missing.reset_index().groupby(['num_cols_missing',
                'percent_cols_missing']).count().reset_index().rename(columns={'index' : 'count'})
    return rows_missing

In [11]:
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    prcnt_miss = num_missing / df.shape[0] * 100
    
    cols_missing = pd.DataFrame({'num_rows_missing' : num_missing,
                                 'percent_rows_missing' : prcnt_miss})
    return cols_missing

In [12]:
nulls_by_col(df)

,num_rows_missing,percent_rows_missing
parcelid,0,0.00
bathrooms,0,0.00
bedrooms,0,0.00
calculatedbathnbr,32,0.06
calculatedfinishedsquarefeet,8,0.02
finishedsquarefeet12,166,0.32
fips,0,0.00
fullbathcnt,32,0.06
latitude,0,0.00
longitude,0,0.00


In [13]:
nulls_by_row(df)

,num_cols_missing,percent_cols_missing,count
0,0,0.00,32130
1,1,3.33,1492
2,2,6.67,645
3,3,10.00,17285
4,4,13.33,546
5,5,16.67,25
6,6,20.00,30
7,7,23.33,4
8,8,26.67,1


In [14]:
def single_family(df):
    df=df[df.propertylandusedesc == 'Single Family Residential']
    df=df[df.unitcnt <= 1]

In [15]:
df.propertylandusedesc.value_counts()

Single Family Residential    52158
Name: propertylandusedesc, dtype: int64

In [16]:
single_family(df)

In [17]:
df.bedrooms.value_counts()

 3.00    23296
 4.00    15202
 2.00     8314
 5.00     3964
 6.00      633
 1.00      610
 7.00      105
 8.00       23
 9.00        8
10.00        2
11.00        1
Name: bedrooms, dtype: int64

In [18]:
df.shape
        

(52158, 30)

In [19]:
def handle_missing_values(df, prop_required_column=.5, prop_required_row=.75):
    threshold = int(round(prop_required_column * len(df.index), 0))
    df = df.dropna(axis=1, thresh = threshold)
    threshold = int(round(prop_required_row * len(df.columns), 0))
    df = df.dropna(axis=0, thresh = threshold)
    return df

In [20]:
df = handle_missing_values(df)

In [21]:
import wrangle_mall
import acquire
import prepare

from sklearn.preprocessing import MinMaxScaler


import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'acquire'

In [ ]:
df = wrangle_mall.get_mallcustomer_data()
df.shape

In [ ]:
df = wrangle_mall.detect_outliers(df)
df.shape

In [ ]:
df = wrangle_mall.mall_dummies(df)
df

In [ ]:
train, validate, test = wrangle_mall.split_mall(df)

In [ ]:
s_train, s_validate, s_test = wrangle_mall.mall_scaler(train, validate, test)

In [ ]:
s_validate